### Implementation of [Simple and Scalable Predictive Uncertainty Estimation using Deep Ensembles](https://papers.nips.cc/paper/7219-simple-and-scalable-predictive-uncertainty-estimation-using-deep-ensembles.pdf)

Note: Kernel died before I could plot the results, so I had to rerun in the last minute to at least show results for M=1 (refer to "Complete Pipeline (M=1)"

In [1]:
from __future__ import print_function
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range
import os
import keras
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import os
import pickle
import random

import load_data
import models

Using TensorFlow backend.


In [2]:
print("notMNIST")
notMNIST_X_train, notMNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test = load_data.data_notMNIST()
print("MNIST")
MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test = load_data.data_MNIST()

notMNIST
20000 train samples
1000 valid samples
1000 test samples
MNIST
Train labels dimension:
(50000, 10)
Test labels dimension:
(10000, 10)


In [3]:
def Ensembling_Data(M, MNIST_X_train, MNIST_Y_train):
#     M = 5
    sampled_indices = np.arange(0,MNIST_X_train.shape[0])
    random.shuffle(sampled_indices)

    cut = int(MNIST_X_train.shape[0]/M)
    index_sample_list = []
    for i in range(M):
        index_sample_list.append(sampled_indices[cut*i:cut*(i+1)])
        
    training_X_df = []; training_Y_df = []
    for index_range in index_sample_list:
        training_X_df.append(MNIST_X_train[[index_range]])
        training_Y_df.append(MNIST_Y_train[[index_range]])
        
    return training_X_df, training_Y_df

def NLL(pred_values, MNIST_Y_test, index_of_y):

    from keras import backend as K

    # generate NLL distribution
#     pred_hotcoded = np_utils.to_categorical(pred_values, 10)[index_of_y:index_of_y+1]

    # y_test = y_test.astype('float32') # necessary here, since y_pred comes in this type - check in your case with y_test.dtype and y_pred.dtype
    # y_test = K.constant(y_test)
    # y_pred = K.constant(y_pred)

#     y_pred = K.constant(pred_hotcoded)
    
    y_pred = K.constant(pred_values[index_of_y:index_of_y+1])

    g = K.categorical_crossentropy(target=MNIST_Y_test[index_of_y:index_of_y+1], output=y_pred)  # tensor
    ce = K.eval(g)  # 'ce' for cross-entropy
    ce.shape
    # (10000,) # i.e. one loss quantity per sample

    # sum up and divide with the no. of samples:
    log_loss = np.sum(ce)/ce.shape[0]
#     log_loss
    # 0.05165323486328125
    
    # https://stackoverflow.com/questions/52497625/how-to-calculate-negative-log-likelihoog-on-mnist-dataset
    return log_loss

def entropy_values(MNIST_Y_test, prediction):
    entropy_values=[]
    # Note: To obtain results faster, we use 200 samples for testing purposes
    for i in range(len(MNIST_Y_test[0:200])): # WARNING: Remove 100 limit, let whole dataframe!!!
        log_loss = NLL(prediction, MNIST_Y_test, i)
        entropy_values.append(log_loss)
#         print(i)
    #     print(log_loss)
    return entropy_values

# def entropy_values_faster(MNIST_Y_test, prediction):
# #     entropy_values=[]
# #     for i in range(len(MNIST_Y_test[0:])): # WARNING: Remove 100 limit, let whole dataframe!!!
# #         log_loss = NLL(prediction, MNIST_Y_test, i)
# #         entropy_values.append(log_loss)
#     entropy_values = [NLL(prediction, MNIST_Y_test, i) for i in range(len(MNIST_Y_test))]
#     #     print(log_loss)
#     return entropy_values



In [4]:
def ensembling_expt(M, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, model_function):
    
    training_X_df, training_Y_df = Ensembling_Data(M, MNIST_X_train, MNIST_Y_train)

    prediction_storage=[]
    acc_storage=[]
    for i in range(M):
        X_train = training_X_df[i]
        Y_train = training_Y_df[i]
#         model_function = models.MLP()
#         pred = models.MLP(X_train, Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test)
        pred, acc = model_function(X_train, Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test)
        prediction_storage.append(pred)
        acc_storage.append(acc)
#     print(prediction_storage[0])

    confi_list=[]
    for item in prediction_storage:
        tmplist=[]
        for array in item:
            confidence = max(array) # apparently this is defined as confidence by the paper??
            tmplist.append(confidence)
        confi_list.append(pd.Series(tmplist).mean())

    avg_predictions_from_ensemble=[]
    for j in range(len(prediction_storage[0])):
        tmpList=[]
        for i in range(M):
            tmpList.append(prediction_storage[i][j])
        pred_avg = np.average(tmpList, axis=0)
        avg_predictions_from_ensemble.append(pred_avg)
    acc_avg = np.average(acc_storage, axis=0)
    avg_predictions_from_ensemble2 = np.array([list(i) for i in avg_predictions_from_ensemble])
    entropy_val = entropy_values(MNIST_Y_test, avg_predictions_from_ensemble2)
    
    return entropy_val, acc_storage, confi_list

## Baseline (MC Dropout)

In [5]:
# M in MC dropout refers to number of MC dropout samples; since we're sampling different sets of indices, sort of sampling~

entropy_val_MLPmc_1, acc_MLPmc_1, confi_MLPmc_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MCDropout)
entropy_val_MLPmc_5, acc_MLPmc_5, confi_MLPmc_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MCDropout)
entropy_val_MLPmc_10, acc_MLPmc_10, confi_MLPmc_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MCDropout)

Nentropy_val_MLPmc_1, Nacc_MLPmc_1, Nconfi_MLPmc_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MCDropout)
Nentropy_val_MLPmc_5, Nacc_MLPmc_5, Nconfi_MLPmc_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MCDropout)
Nentropy_val_MLPmc_10, Nacc_MLPmc_10, Nconfi_MLPmc_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MCDropout)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


With Dropout & Batch Normalization


W0212 05:35:47.378006 139990761948928 module_wrapper.py:139] From /home/root/adversarial/Gal/models.py:158: The name tf.InteractiveSession is deprecated. Please use tf.compat.v1.InteractiveSession instead.

W0212 05:35:49.808238 139990761948928 module_wrapper.py:139] From /home/root/adversarial/Gal/models.py:171: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0212 05:35:49.813311 139990761948928 module_wrapper.py:139] From /home/root/adversarial/Gal/models.py:177: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0212 05:35:49.902889 139990761948928 deprecation.py:506] From /home/root/adversarial/Gal/models.py:192: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0212 05:35:50.013988 139990761948928 module_wrapper.py:139] From /home

Epoch:0 | Train loss: 38.44 | Train acc: 0.949 | Test acc:0.950
Epoch:1 | Train loss: 22.43 | Train acc: 0.966 | Test acc:0.964
Epoch:2 | Train loss: 13.32 | Train acc: 0.974 | Test acc:0.967
Epoch:3 | Train loss: 8.09 | Train acc: 0.979 | Test acc:0.972
Epoch:4 | Train loss: 5.11 | Train acc: 0.982 | Test acc:0.976
Epoch:5 | Train loss: 3.42 | Train acc: 0.985 | Test acc:0.976
Epoch:6 | Train loss: 2.49 | Train acc: 0.986 | Test acc:0.974
Epoch:7 | Train loss: 1.99 | Train acc: 0.987 | Test acc:0.976
Epoch:8 | Train loss: 1.73 | Train acc: 0.988 | Test acc:0.977
Epoch:9 | Train loss: 1.59 | Train acc: 0.989 | Test acc:0.979
Epoch:10 | Train loss: 1.53 | Train acc: 0.989 | Test acc:0.978
Epoch:11 | Train loss: 1.50 | Train acc: 0.991 | Test acc:0.979
Epoch:12 | Train loss: 1.48 | Train acc: 0.991 | Test acc:0.980
Epoch:13 | Train loss: 1.48 | Train acc: 0.991 | Test acc:0.979


W0212 05:36:47.990037 139990761948928 module_wrapper.py:139] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0212 05:36:47.995147 139990761948928 module_wrapper.py:139] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0212 05:36:47.996059 139990761948928 module_wrapper.py:139] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0212 05:36:47.997046 139990761948928 module_wrapper.py:139] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

/usr/local/lib/python3.5/dist-packages/ipykern

...

Epoch:0 | Train loss: 50.78 | Train acc: 0.762 | Test acc:0.066
Epoch:1 | Train loss: 47.42 | Train acc: 0.881 | Test acc:0.113
Epoch:2 | Train loss: 44.39 | Train acc: 0.914 | Test acc:0.107
Epoch:3 | Train loss: 41.60 | Train acc: 0.939 | Test acc:0.114
Epoch:4 | Train loss: 39.01 | Train acc: 0.950 | Test acc:0.128
Epoch:5 | Train loss: 36.60 | Train acc: 0.956 | Test acc:0.132
Epoch:6 | Train loss: 34.35 | Train acc: 0.961 | Test acc:0.124
Epoch:7 | Train loss: 32.26 | Train acc: 0.966 | Test acc:0.114
Epoch:8 | Train loss: 30.30 | Train acc: 0.971 | Test acc:0.122
Epoch:9 | Train loss: 28.47 | Train acc: 0.973 | Test acc:0.135
Epoch:10 | Train loss: 26.76 | Train acc: 0.980 | Test acc:0.136
Epoch:11 | Train loss: 25.16 | Train acc: 0.980 | Test acc:0.141
Epoch:12 | Train loss: 23.66 | Train acc: 0.983 | Test acc:0.130
Epoch:13 | Train loss: 22.25 | Train acc: 0.983 | Test acc:0.115
With Dropout & Batch Normalization


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 56.85 | Train acc: 0.804 | Test acc:0.134
Epoch:1 | Train loss: 53.32 | Train acc: 0.886 | Test acc:0.138
Epoch:2 | Train loss: 50.09 | Train acc: 0.914 | Test acc:0.129
Epoch:3 | Train loss: 47.11 | Train acc: 0.933 | Test acc:0.137
Epoch:4 | Train loss: 44.33 | Train acc: 0.942 | Test acc:0.121
Epoch:5 | Train loss: 41.73 | Train acc: 0.950 | Test acc:0.109
Epoch:6 | Train loss: 39.30 | Train acc: 0.959 | Test acc:0.104
Epoch:7 | Train loss: 37.02 | Train acc: 0.968 | Test acc:0.110
Epoch:8 | Train loss: 34.89 | Train acc: 0.968 | Test acc:0.083
Epoch:9 | Train loss: 32.89 | Train acc: 0.970 | Test acc:0.077
Epoch:10 | Train loss: 31.01 | Train acc: 0.975 | Test acc:0.071
Epoch:11 | Train loss: 29.24 | Train acc: 0.981 | Test acc:0.059
Epoch:12 | Train loss: 27.58 | Train acc: 0.983 | Test acc:0.071
Epoch:13 | Train loss: 26.02 | Train acc: 0.984 | Test acc:0.056
With Dropout & Batch Normalization


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 59.17 | Train acc: 0.823 | Test acc:0.124
Epoch:1 | Train loss: 55.44 | Train acc: 0.881 | Test acc:0.100
Epoch:2 | Train loss: 52.04 | Train acc: 0.905 | Test acc:0.109
Epoch:3 | Train loss: 48.89 | Train acc: 0.924 | Test acc:0.126
Epoch:4 | Train loss: 45.96 | Train acc: 0.933 | Test acc:0.102
Epoch:5 | Train loss: 43.23 | Train acc: 0.943 | Test acc:0.111
Epoch:6 | Train loss: 40.67 | Train acc: 0.951 | Test acc:0.107
Epoch:7 | Train loss: 38.27 | Train acc: 0.958 | Test acc:0.103
Epoch:8 | Train loss: 36.02 | Train acc: 0.966 | Test acc:0.098
Epoch:9 | Train loss: 33.91 | Train acc: 0.970 | Test acc:0.124
Epoch:10 | Train loss: 31.93 | Train acc: 0.974 | Test acc:0.121
Epoch:11 | Train loss: 30.07 | Train acc: 0.980 | Test acc:0.120
Epoch:12 | Train loss: 28.32 | Train acc: 0.983 | Test acc:0.121
Epoch:13 | Train loss: 26.69 | Train acc: 0.985 | Test acc:0.119


## Ensembling MLP & variants

### MNIST plots

In [6]:
entropy_val_MLP_1, acc_MLP_1, confi_MLP_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP)
entropy_val_MLP_5, acc_MLP_5, confi_MLP_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP)
entropy_val_MLP_10, acc_MLP_10, confi_MLP_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

Without Dropout, Vanilla MLP
Epoch:0 | Train loss: 34.49 | Train acc: 0.938 | Test acc:0.938
Epoch:1 | Train loss: 19.25 | Train acc: 0.955 | Test acc:0.952
Epoch:2 | Train loss: 10.95 | Train acc: 0.968 | Test acc:0.963
Epoch:3 | Train loss: 6.44 | Train acc: 0.976 | Test acc:0.967
Epoch:4 | Train loss: 4.00 | Train acc: 0.978 | Test acc:0.968
Epoch:5 | Train loss: 2.70 | Train acc: 0.980 | Test acc:0.967
Epoch:6 | Train loss: 2.04 | Train acc: 0.983 | Test acc:0.972
Epoch:7 | Train loss: 1.71 | Train acc: 0.987 | Test acc:0.973
Epoch:8 | Train loss: 1.56 | Train acc: 0.987 | Test acc:0.973
Epoch:9 | Train loss: 1.50 | Train acc: 0.988 | Test acc:0.973
Epoch:10 | Train loss: 1.48 | Train acc: 0.988 | Test acc:0.973
Epoch:11 | Train loss: 1.47 | Train acc: 0.991 | Test acc:0.976
Epoch:12 | Train loss: 1.47 | Train acc: 0.993 | Test acc:0.978
Epoch:13 | Train loss: 1.47 | Train acc: 0.993 | Test acc:0.977


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

Without Dropout, Vanilla MLP
Epoch:0 | Train loss: 55.09 | Train acc: 0.879 | Test acc:0.882
Epoch:1 | Train loss: 48.84 | Train acc: 0.898 | Test acc:0.892
Epoch:2 | Train loss: 43.36 | Train acc: 0.928 | Test acc:0.919
Epoch:3 | Train loss: 38.55 | Train acc: 0.937 | Test acc:0.924
Epoch:4 | Train loss: 34.31 | Train acc: 0.951 | Test acc:0.935
Epoch:5 | Train loss: 30.56 | Train acc: 0.959 | Test acc:0.936
Epoch:6 | Train loss: 27.24 | Train acc: 0.965 | Test acc:0.940
Epoch:7 | Train loss: 24.29 | Train acc: 0.971 | Test acc:0.945
Epoch:8 | Train loss: 21.68 | Train acc: 0.973 | Test acc:0.947
Epoch:9 | Train loss: 19.36 | Train acc: 0.973 | Test acc:0.945
Epoch:10 | Train loss: 17.30 | Train acc: 0.979 | Test acc:0.953
Epoch:11 | Train loss: 15.46 | Train acc: 0.982 | Test acc:0.951
Epoch:12 | Train loss: 13.83 | Train acc: 0.985 | Test acc:0.956
Epoch:13 | Train loss: 12.39 | Train acc: 0.986 | Test acc:0.955
Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


...

Epoch:0 | Train loss: 57.65 | Train acc: 0.797 | Test acc:0.808
Epoch:1 | Train loss: 54.03 | Train acc: 0.848 | Test acc:0.839
Epoch:2 | Train loss: 50.70 | Train acc: 0.894 | Test acc:0.887
Epoch:3 | Train loss: 47.62 | Train acc: 0.897 | Test acc:0.890
Epoch:4 | Train loss: 44.73 | Train acc: 0.926 | Test acc:0.914
Epoch:5 | Train loss: 42.03 | Train acc: 0.933 | Test acc:0.917
Epoch:6 | Train loss: 39.51 | Train acc: 0.943 | Test acc:0.925
Epoch:7 | Train loss: 37.15 | Train acc: 0.951 | Test acc:0.928
Epoch:8 | Train loss: 34.95 | Train acc: 0.950 | Test acc:0.924
Epoch:9 | Train loss: 32.87 | Train acc: 0.958 | Test acc:0.928
Epoch:10 | Train loss: 30.92 | Train acc: 0.961 | Test acc:0.930
Epoch:11 | Train loss: 29.10 | Train acc: 0.966 | Test acc:0.934
Epoch:12 | Train loss: 27.38 | Train acc: 0.970 | Test acc:0.938
Epoch:13 | Train loss: 25.78 | Train acc: 0.974 | Test acc:0.936
Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 58.17 | Train acc: 0.832 | Test acc:0.829
Epoch:1 | Train loss: 54.64 | Train acc: 0.869 | Test acc:0.858
Epoch:2 | Train loss: 51.41 | Train acc: 0.892 | Test acc:0.870
Epoch:3 | Train loss: 48.42 | Train acc: 0.912 | Test acc:0.899
Epoch:4 | Train loss: 45.61 | Train acc: 0.928 | Test acc:0.907
Epoch:5 | Train loss: 42.98 | Train acc: 0.947 | Test acc:0.916
Epoch:6 | Train loss: 40.52 | Train acc: 0.951 | Test acc:0.920
Epoch:7 | Train loss: 38.21 | Train acc: 0.957 | Test acc:0.923
Epoch:8 | Train loss: 36.04 | Train acc: 0.959 | Test acc:0.920
Epoch:9 | Train loss: 34.00 | Train acc: 0.966 | Test acc:0.928
Epoch:10 | Train loss: 32.08 | Train acc: 0.971 | Test acc:0.930
Epoch:11 | Train loss: 30.28 | Train acc: 0.972 | Test acc:0.930
Epoch:12 | Train loss: 28.58 | Train acc: 0.973 | Test acc:0.932
Epoch:13 | Train loss: 26.98 | Train acc: 0.976 | Test acc:0.935
Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 57.83 | Train acc: 0.791 | Test acc:0.803
Epoch:1 | Train loss: 54.28 | Train acc: 0.844 | Test acc:0.840
Epoch:2 | Train loss: 51.03 | Train acc: 0.864 | Test acc:0.852
Epoch:3 | Train loss: 48.00 | Train acc: 0.900 | Test acc:0.889
Epoch:4 | Train loss: 45.17 | Train acc: 0.914 | Test acc:0.893
Epoch:5 | Train loss: 42.52 | Train acc: 0.924 | Test acc:0.904
Epoch:6 | Train loss: 40.04 | Train acc: 0.938 | Test acc:0.919
Epoch:7 | Train loss: 37.71 | Train acc: 0.945 | Test acc:0.921
Epoch:8 | Train loss: 35.52 | Train acc: 0.948 | Test acc:0.925
Epoch:9 | Train loss: 33.47 | Train acc: 0.955 | Test acc:0.928
Epoch:10 | Train loss: 31.54 | Train acc: 0.960 | Test acc:0.930
Epoch:11 | Train loss: 29.73 | Train acc: 0.963 | Test acc:0.930
Epoch:12 | Train loss: 28.02 | Train acc: 0.966 | Test acc:0.930
Epoch:13 | Train loss: 26.42 | Train acc: 0.966 | Test acc:0.929


In [7]:
entropy_val_MLPr_1, acc_MLPr_1, confi_MLPr_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_randomperturbation)
entropy_val_MLPr_5, acc_MLPr_5, confi_MLPr_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_randomperturbation)
entropy_val_MLPr_10, acc_MLPr_10, confi_MLPr_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_randomperturbation)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Without Dropout, MLP with random perturbations


/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((60000, 784), (60000, 10))
Epoch:0 | Train loss: 29.70 | Train acc: 0.909 | Test acc:0.908
Epoch:1 | Train loss: 15.21 | Train acc: 0.937 | Test acc:0.933
Epoch:2 | Train loss: 7.98 | Train acc: 0.919 | Test acc:0.911
Epoch:3 | Train loss: 4.38 | Train acc: 0.934 | Test acc:0.933
Epoch:4 | Train loss: 2.68 | Train acc: 0.942 | Test acc:0.932
Epoch:5 | Train loss: 1.92 | Train acc: 0.974 | Test acc:0.965
Epoch:6 | Train loss: 1.62 | Train acc: 0.983 | Test acc:0.971
Epoch:7 | Train loss: 1.52 | Train acc: 0.978 | Test acc:0.966
Epoch:8 | Train loss: 1.49 | Train acc: 0.976 | Test acc:0.965
Epoch:9 | Train loss: 1.49 | Train acc: 0.970 | Test acc:0.957
Epoch:10 | Train loss: 1.47 | Train acc: 0.987 | Test acc:0.972
Epoch:11 | Train loss: 1.48 | Train acc: 0.972 | Test acc:0.958
Epoch:12 | Train loss: 1.47 | Train acc: 0.993 | Test acc:0.977
Epoch:13 | Train loss: 1.51 | Train acc: 0.931 | Test acc:0.918


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different

...

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 52.15 | Train acc: 0.456 | Test acc:0.460
Epoch:1 | Train loss: 48.25 | Train acc: 0.831 | Test acc:0.837
Epoch:2 | Train loss: 44.81 | Train acc: 0.797 | Test acc:0.793
Epoch:3 | Train loss: 41.69 | Train acc: 0.779 | Test acc:0.768
Epoch:4 | Train loss: 38.76 | Train acc: 0.838 | Test acc:0.822
Epoch:5 | Train loss: 36.02 | Train acc: 0.923 | Test acc:0.907
Epoch:6 | Train loss: 33.54 | Train acc: 0.931 | Test acc:0.911
Epoch:7 | Train loss: 31.24 | Train acc: 0.934 | Test acc:0.911
Epoch:8 | Train loss: 29.13 | Train acc: 0.914 | Test acc:0.892
Epoch:9 | Train loss: 27.14 | Train acc: 0.946 | Test acc:0.914
Epoch:10 | Train loss: 25.31 | Train acc: 0.950 | Test acc:0.918
Epoch:11 | Train loss: 23.67 | Train acc: 0.867 | Test acc:0.843
Epoch:12 | Train loss: 22.08 | Train acc: 0.904 | Test acc:0.873
Epoch:13 | Train loss: 20.57 | Train acc: 0.960 | Test acc:0.929
Without Dropout, MLP with random perturbati

/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 53.56 | Train acc: 0.428 | Test acc:0.443
Epoch:1 | Train loss: 49.53 | Train acc: 0.771 | Test acc:0.773
Epoch:2 | Train loss: 45.90 | Train acc: 0.858 | Test acc:0.856
Epoch:3 | Train loss: 42.57 | Train acc: 0.921 | Test acc:0.904
Epoch:4 | Train loss: 39.54 | Train acc: 0.895 | Test acc:0.882
Epoch:5 | Train loss: 36.74 | Train acc: 0.889 | Test acc:0.874
Epoch:6 | Train loss: 34.15 | Train acc: 0.876 | Test acc:0.859
Epoch:7 | Train loss: 31.69 | Train acc: 0.956 | Test acc:0.922
Epoch:8 | Train loss: 29.45 | Train acc: 0.959 | Test acc:0.922
Epoch:9 | Train loss: 27.38 | Train acc: 0.966 | Test acc:0.928
Epoch:10 | Train loss: 25.46 | Train acc: 0.974 | Test acc:0.934
Epoch:11 | Train loss: 23.67 | Train acc: 0.970 | Test acc:0.927
Epoch:12 | Train loss: 22.02 | Train acc: 0.973 | Test acc:0.932
Epoch:13 | Train loss: 20.48 | Train acc: 0.976 | Test acc:0.931
Without Dropout, MLP with random perturbati

/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 57.93 | Train acc: 0.511 | Test acc:0.511
Epoch:1 | Train loss: 53.80 | Train acc: 0.721 | Test acc:0.715
Epoch:2 | Train loss: 49.98 | Train acc: 0.883 | Test acc:0.870
Epoch:3 | Train loss: 46.53 | Train acc: 0.913 | Test acc:0.898
Epoch:4 | Train loss: 43.42 | Train acc: 0.784 | Test acc:0.776
Epoch:5 | Train loss: 40.40 | Train acc: 0.937 | Test acc:0.910
Epoch:6 | Train loss: 37.68 | Train acc: 0.944 | Test acc:0.916
Epoch:7 | Train loss: 35.14 | Train acc: 0.949 | Test acc:0.917
Epoch:8 | Train loss: 32.79 | Train acc: 0.951 | Test acc:0.918
Epoch:9 | Train loss: 30.61 | Train acc: 0.949 | Test acc:0.920
Epoch:10 | Train loss: 28.58 | Train acc: 0.927 | Test acc:0.893
Epoch:11 | Train loss: 26.67 | Train acc: 0.970 | Test acc:0.930
Epoch:12 | Train loss: 24.92 | Train acc: 0.947 | Test acc:0.912
Epoch:13 | Train loss: 23.31 | Train acc: 0.894 | Test acc:0.859


In [ ]:
entropy_val_MLPat_1, acc_MLPat_1, confi_MLPat_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_adversarialtraining)
entropy_val_MLPat_5, acc_MLPat_5, confi_MLPat_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_adversarialtraining)
entropy_val_MLPat_10, acc_MLPat_10, confi_MLPat_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, MNIST_X_val, MNIST_Y_val, MNIST_X_test, MNIST_Y_test, models.MLP_adversarialtraining)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

Without Dropout & Batch Normalization
Epoch:0 | Train loss: 33.96 | Train acc: 0.973 | Test acc:0.967
Epoch:1 | Train loss: 19.60 | Train acc: 0.982 | Test acc:0.974
Epoch:2 | Train loss: 11.53 | Train acc: 0.988 | Test acc:0.979
Epoch:3 | Train loss: 6.95 | Train acc: 0.990 | Test acc:0.976
Epoch:4 | Train loss: 4.37 | Train acc: 0.993 | Test acc:0.979
Epoch:5 | Train loss: 2.95 | Train acc: 0.994 | Test acc:0.979
Epoch:6 | Train loss: 2.20 | Train acc: 0.995 | Test acc:0.980
Epoch:7 | Train loss: 1.81 | Train acc: 0.996 | Test acc:0.981
Epoch:8 | Train loss: 1.62 | Train acc: 0.997 | Test acc:0.982
Epoch:9 | Train loss: 1.53 | Train acc: 0.997 | Test acc:0.982
Epoch:10 | Train loss: 1.49 | Train acc: 0.997 | Test acc:0.980
Epoch:11 | Train loss: 1.47 | Train acc: 0.998 | Test acc:0.982
Epoch:12 | Train loss: 1.47 | Train acc: 0.997 | Test acc:0.980
Epoch:13 | Train loss: 1.47 | Train acc: 0.997 | Test acc:0.980
Without Dropout & Batch Normalization & Adversarial Training


/home/root/adversarial/Gal/models.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:422: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

Epoch:0 | Train loss: 34.19 | Train acc: 0.830 | Test acc:0.968
Epoch:1 | Train loss: 17.68 | Train acc: 0.837 | Test acc:0.974
Epoch:2 | Train loss: 9.31 | Train acc: 0.843 | Test acc:0.979
Epoch:3 | Train loss: 5.12 | Train acc: 0.845 | Test acc:0.981
Epoch:4 | Train loss: 3.09 | Train acc: 0.843 | Test acc:0.980
Epoch:5 | Train loss: 2.15 | Train acc: 0.846 | Test acc:0.981
Epoch:6 | Train loss: 1.73 | Train acc: 0.846 | Test acc:0.981
Epoch:7 | Train loss: 1.56 | Train acc: 0.846 | Test acc:0.981
Epoch:8 | Train loss: 1.50 | Train acc: 0.845 | Test acc:0.978
Epoch:9 | Train loss: 1.47 | Train acc: 0.849 | Test acc:0.982
Epoch:10 | Train loss: 1.47 | Train acc: 0.848 | Test acc:0.983
Epoch:11 | Train loss: 1.46 | Train acc: 0.848 | Test acc:0.981
Epoch:12 | Train loss: 1.46 | Train acc: 0.847 | Test acc:0.981
Epoch:13 | Train loss: 1.46 | Train acc: 0.848 | Test acc:0.981


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

...

Epoch:0 | Train loss: 57.85 | Train acc: 0.779 | Test acc:0.896
Epoch:1 | Train loss: 53.84 | Train acc: 0.825 | Test acc:0.937
Epoch:2 | Train loss: 50.21 | Train acc: 0.839 | Test acc:0.947
Epoch:3 | Train loss: 46.86 | Train acc: 0.843 | Test acc:0.948
Epoch:4 | Train loss: 43.77 | Train acc: 0.845 | Test acc:0.953
Epoch:5 | Train loss: 40.91 | Train acc: 0.847 | Test acc:0.952
Epoch:6 | Train loss: 38.26 | Train acc: 0.848 | Test acc:0.952
Epoch:7 | Train loss: 35.79 | Train acc: 0.850 | Test acc:0.956
Epoch:8 | Train loss: 33.50 | Train acc: 0.852 | Test acc:0.956
Epoch:9 | Train loss: 31.36 | Train acc: 0.849 | Test acc:0.956
Epoch:10 | Train loss: 29.38 | Train acc: 0.845 | Test acc:0.956
Epoch:11 | Train loss: 27.52 | Train acc: 0.849 | Test acc:0.957
Epoch:12 | Train loss: 25.80 | Train acc: 0.849 | Test acc:0.957
Epoch:13 | Train loss: 24.19 | Train acc: 0.850 | Test acc:0.958
Without Dropout & Batch Normalization


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 55.29 | Train acc: 0.920 | Test acc:0.900
Epoch:1 | Train loss: 51.80 | Train acc: 0.958 | Test acc:0.931
Epoch:2 | Train loss: 48.61 | Train acc: 0.975 | Test acc:0.938
Epoch:3 | Train loss: 45.65 | Train acc: 0.985 | Test acc:0.945
Epoch:4 | Train loss: 42.88 | Train acc: 0.990 | Test acc:0.947
Epoch:5 | Train loss: 40.30 | Train acc: 0.993 | Test acc:0.950
Epoch:6 | Train loss: 37.89 | Train acc: 0.994 | Test acc:0.951
Epoch:7 | Train loss: 35.63 | Train acc: 0.996 | Test acc:0.949
Epoch:8 | Train loss: 33.51 | Train acc: 0.995 | Test acc:0.946
Epoch:9 | Train loss: 31.52 | Train acc: 0.996 | Test acc:0.947
Epoch:10 | Train loss: 29.66 | Train acc: 0.996 | Test acc:0.947
Epoch:11 | Train loss: 27.91 | Train acc: 0.997 | Test acc:0.950
Epoch:12 | Train loss: 26.27 | Train acc: 0.996 | Test acc:0.947
Epoch:13 | Train loss: 24.73 | Train acc: 0.997 | Test acc:0.946
Without Dropout & Batch Normalization & Adversarial Training


/home/root/adversarial/Gal/models.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:422: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

Epoch:0 | Train loss: 58.92 | Train acc: 0.800 | Test acc:0.914
Epoch:1 | Train loss: 54.69 | Train acc: 0.831 | Test acc:0.935
Epoch:2 | Train loss: 50.83 | Train acc: 0.841 | Test acc:0.941
Epoch:3 | Train loss: 47.28 | Train acc: 0.846 | Test acc:0.945
Epoch:4 | Train loss: 43.99 | Train acc: 0.848 | Test acc:0.947
Epoch:5 | Train loss: 40.96 | Train acc: 0.850 | Test acc:0.948
Epoch:6 | Train loss: 38.14 | Train acc: 0.850 | Test acc:0.951
Epoch:7 | Train loss: 35.53 | Train acc: 0.850 | Test acc:0.948
Epoch:8 | Train loss: 33.11 | Train acc: 0.848 | Test acc:0.953
Epoch:9 | Train loss: 30.86 | Train acc: 0.848 | Test acc:0.954
Epoch:10 | Train loss: 28.77 | Train acc: 0.848 | Test acc:0.951
Epoch:11 | Train loss: 26.82 | Train acc: 0.851 | Test acc:0.952
Epoch:12 | Train loss: 25.02 | Train acc: 0.849 | Test acc:0.954
Epoch:13 | Train loss: 23.34 | Train acc: 0.850 | Test acc:0.954


### notMNIST plots

In [ ]:
Nentropy_val_MLP_1, Nacc_MLP_1, Nconfi_MLP_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP)
Nentropy_val_MLP_5, Nacc_MLP_5, Nconfi_MLP_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP)
Nentropy_val_MLP_10, Nacc_MLP_10, Nconfi_MLP_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

Without Dropout, Vanilla MLP
Epoch:0 | Train loss: 33.88 | Train acc: 0.937 | Test acc:0.064
Epoch:1 | Train loss: 19.04 | Train acc: 0.958 | Test acc:0.075
Epoch:2 | Train loss: 10.93 | Train acc: 0.967 | Test acc:0.056
Epoch:3 | Train loss: 6.47 | Train acc: 0.974 | Test acc:0.085
Epoch:4 | Train loss: 4.04 | Train acc: 0.980 | Test acc:0.072
Epoch:5 | Train loss: 2.74 | Train acc: 0.983 | Test acc:0.061
Epoch:6 | Train loss: 2.06 | Train acc: 0.985 | Test acc:0.069
Epoch:7 | Train loss: 1.73 | Train acc: 0.983 | Test acc:0.067
Epoch:8 | Train loss: 1.57 | Train acc: 0.987 | Test acc:0.070
Epoch:9 | Train loss: 1.51 | Train acc: 0.989 | Test acc:0.078
Epoch:10 | Train loss: 1.48 | Train acc: 0.992 | Test acc:0.070
Epoch:11 | Train loss: 1.47 | Train acc: 0.988 | Test acc:0.075
Epoch:12 | Train loss: 1.47 | Train acc: 0.991 | Test acc:0.076
Epoch:13 | Train loss: 1.47 | Train acc: 0.992 | Test acc:0.080


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


...

Epoch:0 | Train loss: 57.47 | Train acc: 0.787 | Test acc:0.065
Epoch:1 | Train loss: 53.88 | Train acc: 0.858 | Test acc:0.068
Epoch:2 | Train loss: 50.60 | Train acc: 0.860 | Test acc:0.078
Epoch:3 | Train loss: 47.56 | Train acc: 0.886 | Test acc:0.078
Epoch:4 | Train loss: 44.70 | Train acc: 0.928 | Test acc:0.075
Epoch:5 | Train loss: 42.04 | Train acc: 0.935 | Test acc:0.074
Epoch:6 | Train loss: 39.56 | Train acc: 0.947 | Test acc:0.073
Epoch:7 | Train loss: 37.23 | Train acc: 0.951 | Test acc:0.076
Epoch:8 | Train loss: 35.05 | Train acc: 0.958 | Test acc:0.080
Epoch:9 | Train loss: 33.00 | Train acc: 0.960 | Test acc:0.075
Epoch:10 | Train loss: 31.09 | Train acc: 0.960 | Test acc:0.088
Epoch:11 | Train loss: 29.28 | Train acc: 0.968 | Test acc:0.088
Epoch:12 | Train loss: 27.60 | Train acc: 0.971 | Test acc:0.083
Epoch:13 | Train loss: 26.01 | Train acc: 0.973 | Test acc:0.086
Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 59.47 | Train acc: 0.836 | Test acc:0.052
Epoch:1 | Train loss: 55.81 | Train acc: 0.847 | Test acc:0.080
Epoch:2 | Train loss: 52.46 | Train acc: 0.882 | Test acc:0.086
Epoch:3 | Train loss: 49.32 | Train acc: 0.909 | Test acc:0.086
Epoch:4 | Train loss: 46.40 | Train acc: 0.926 | Test acc:0.088
Epoch:5 | Train loss: 43.67 | Train acc: 0.938 | Test acc:0.084
Epoch:6 | Train loss: 41.10 | Train acc: 0.950 | Test acc:0.085
Epoch:7 | Train loss: 38.70 | Train acc: 0.946 | Test acc:0.080
Epoch:8 | Train loss: 36.44 | Train acc: 0.960 | Test acc:0.082
Epoch:9 | Train loss: 34.32 | Train acc: 0.965 | Test acc:0.077
Epoch:10 | Train loss: 32.34 | Train acc: 0.967 | Test acc:0.090
Epoch:11 | Train loss: 30.47 | Train acc: 0.971 | Test acc:0.080
Epoch:12 | Train loss: 28.71 | Train acc: 0.973 | Test acc:0.084
Epoch:13 | Train loss: 27.06 | Train acc: 0.970 | Test acc:0.069
Without Dropout, Vanilla MLP


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 61.39 | Train acc: 0.817 | Test acc:0.064
Epoch:1 | Train loss: 57.68 | Train acc: 0.870 | Test acc:0.056
Epoch:2 | Train loss: 54.26 | Train acc: 0.898 | Test acc:0.042
Epoch:3 | Train loss: 51.07 | Train acc: 0.920 | Test acc:0.062
Epoch:4 | Train loss: 48.09 | Train acc: 0.928 | Test acc:0.072
Epoch:5 | Train loss: 45.29 | Train acc: 0.943 | Test acc:0.060
Epoch:6 | Train loss: 42.67 | Train acc: 0.948 | Test acc:0.061
Epoch:7 | Train loss: 40.22 | Train acc: 0.947 | Test acc:0.063
Epoch:8 | Train loss: 37.91 | Train acc: 0.958 | Test acc:0.055
Epoch:9 | Train loss: 35.74 | Train acc: 0.955 | Test acc:0.062
Epoch:10 | Train loss: 33.70 | Train acc: 0.966 | Test acc:0.042
Epoch:11 | Train loss: 31.78 | Train acc: 0.968 | Test acc:0.053
Epoch:12 | Train loss: 29.98 | Train acc: 0.969 | Test acc:0.041
Epoch:13 | Train loss: 28.28 | Train acc: 0.971 | Test acc:0.046


In [ ]:
Nentropy_val_MLPr_1, Nacc_MLPr_1, Nconfi_MLPr_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_randomperturbation)
Nentropy_val_MLPr_5, Nacc_MLPr_5, Nconfi_MLPr_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_randomperturbation)
Nentropy_val_MLPr_10, Nacc_MLPr_10, Nconfi_MLPr_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_randomperturbation)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Without Dropout, MLP with random perturbations


/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((60000, 784), (60000, 10))
Epoch:0 | Train loss: 29.11 | Train acc: 0.757 | Test acc:0.094
Epoch:1 | Train loss: 14.90 | Train acc: 0.917 | Test acc:0.065
Epoch:2 | Train loss: 7.93 | Train acc: 0.946 | Test acc:0.054
Epoch:3 | Train loss: 4.47 | Train acc: 0.971 | Test acc:0.091
Epoch:4 | Train loss: 2.81 | Train acc: 0.978 | Test acc:0.086
Epoch:5 | Train loss: 2.04 | Train acc: 0.973 | Test acc:0.089
Epoch:6 | Train loss: 1.70 | Train acc: 0.976 | Test acc:0.086
Epoch:7 | Train loss: 1.56 | Train acc: 0.976 | Test acc:0.107
Epoch:8 | Train loss: 1.50 | Train acc: 0.984 | Test acc:0.094
Epoch:9 | Train loss: 1.48 | Train acc: 0.979 | Test acc:0.089
Epoch:10 | Train loss: 1.47 | Train acc: 0.989 | Test acc:0.102
Epoch:11 | Train loss: 1.47 | Train acc: 0.983 | Test acc:0.110
Epoch:12 | Train loss: 1.51 | Train acc: 0.935 | Test acc:0.068
Epoch:13 | Train loss: 1.47 | Train acc: 0.991 | Test acc:0.083


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different

...

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 52.29 | Train acc: 0.702 | Test acc:0.120
Epoch:1 | Train loss: 48.37 | Train acc: 0.806 | Test acc:0.042
Epoch:2 | Train loss: 44.88 | Train acc: 0.832 | Test acc:0.080
Epoch:3 | Train loss: 41.67 | Train acc: 0.818 | Test acc:0.057
Epoch:4 | Train loss: 38.67 | Train acc: 0.854 | Test acc:0.072
Epoch:5 | Train loss: 35.91 | Train acc: 0.895 | Test acc:0.083
Epoch:6 | Train loss: 33.36 | Train acc: 0.909 | Test acc:0.055
Epoch:7 | Train loss: 31.00 | Train acc: 0.936 | Test acc:0.059
Epoch:8 | Train loss: 28.83 | Train acc: 0.950 | Test acc:0.067
Epoch:9 | Train loss: 26.82 | Train acc: 0.958 | Test acc:0.060
Epoch:10 | Train loss: 24.96 | Train acc: 0.957 | Test acc:0.061
Epoch:11 | Train loss: 23.22 | Train acc: 0.969 | Test acc:0.049
Epoch:12 | Train loss: 21.63 | Train acc: 0.973 | Test acc:0.052
Epoch:13 | Train loss: 20.14 | Train acc: 0.970 | Test acc:0.052
Without Dropout, MLP with random perturbati

/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 53.29 | Train acc: 0.706 | Test acc:0.056
Epoch:1 | Train loss: 49.25 | Train acc: 0.809 | Test acc:0.063
Epoch:2 | Train loss: 45.60 | Train acc: 0.799 | Test acc:0.057
Epoch:3 | Train loss: 42.19 | Train acc: 0.881 | Test acc:0.058
Epoch:4 | Train loss: 39.08 | Train acc: 0.929 | Test acc:0.070
Epoch:5 | Train loss: 36.24 | Train acc: 0.938 | Test acc:0.065
Epoch:6 | Train loss: 33.68 | Train acc: 0.875 | Test acc:0.069
Epoch:7 | Train loss: 31.19 | Train acc: 0.948 | Test acc:0.081
Epoch:8 | Train loss: 28.95 | Train acc: 0.947 | Test acc:0.071
Epoch:9 | Train loss: 26.93 | Train acc: 0.854 | Test acc:0.083
Epoch:10 | Train loss: 24.98 | Train acc: 0.928 | Test acc:0.076
Epoch:11 | Train loss: 23.21 | Train acc: 0.922 | Test acc:0.076
Epoch:12 | Train loss: 21.53 | Train acc: 0.975 | Test acc:0.076
Epoch:13 | Train loss: 20.01 | Train acc: 0.972 | Test acc:0.081
Without Dropout, MLP with random perturbati

/home/root/adversarial/Gal/models.py:656: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:657: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

New training set shape (X,Y):  ((6000, 784), (6000, 10))
Epoch:0 | Train loss: 59.65 | Train acc: 0.458 | Test acc:0.081
Epoch:1 | Train loss: 55.52 | Train acc: 0.744 | Test acc:0.092
Epoch:2 | Train loss: 51.78 | Train acc: 0.780 | Test acc:0.045
Epoch:3 | Train loss: 48.29 | Train acc: 0.904 | Test acc:0.062
Epoch:4 | Train loss: 45.14 | Train acc: 0.890 | Test acc:0.059
Epoch:5 | Train loss: 42.23 | Train acc: 0.831 | Test acc:0.079
Epoch:6 | Train loss: 39.45 | Train acc: 0.944 | Test acc:0.053
Epoch:7 | Train loss: 36.94 | Train acc: 0.906 | Test acc:0.048
Epoch:8 | Train loss: 34.56 | Train acc: 0.960 | Test acc:0.060
Epoch:9 | Train loss: 32.39 | Train acc: 0.933 | Test acc:0.063
Epoch:10 | Train loss: 30.32 | Train acc: 0.970 | Test acc:0.046
Epoch:11 | Train loss: 28.42 | Train acc: 0.967 | Test acc:0.044
Epoch:12 | Train loss: 26.65 | Train acc: 0.974 | Test acc:0.052
Epoch:13 | Train loss: 24.99 | Train acc: 0.979 | Test acc:0.061


In [ ]:
Nentropy_val_MLPat_1, Nacc_MLPat_1, Nconfi_MLPat_1 = ensembling_expt(1, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_adversarialtraining)
Nentropy_val_MLPat_5, Nacc_MLPat_5, Nconfi_MLPat_5 = ensembling_expt(5, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_adversarialtraining)
Nentropy_val_MLPat_10, Nacc_MLPat_10, Nconfi_MLPat_10 = ensembling_expt(10, MNIST_X_train, MNIST_Y_train, notMNIST_X_val, notMNIST_Y_val, notMNIST_X_test, notMNIST_Y_test, models.MLP_adversarialtraining)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

Without Dropout & Batch Normalization
Epoch:0 | Train loss: 30.81 | Train acc: 0.974 | Test acc:0.105
Epoch:1 | Train loss: 17.02 | Train acc: 0.985 | Test acc:0.125
Epoch:2 | Train loss: 9.61 | Train acc: 0.990 | Test acc:0.125
Epoch:3 | Train loss: 5.63 | Train acc: 0.991 | Test acc:0.122
Epoch:4 | Train loss: 3.52 | Train acc: 0.992 | Test acc:0.121
Epoch:5 | Train loss: 2.44 | Train acc: 0.995 | Test acc:0.103
Epoch:6 | Train loss: 1.90 | Train acc: 0.995 | Test acc:0.090
Epoch:7 | Train loss: 1.65 | Train acc: 0.996 | Test acc:0.081
Epoch:8 | Train loss: 1.54 | Train acc: 0.995 | Test acc:0.102
Epoch:9 | Train loss: 1.49 | Train acc: 0.997 | Test acc:0.099
Epoch:10 | Train loss: 1.47 | Train acc: 0.997 | Test acc:0.108
Epoch:11 | Train loss: 1.47 | Train acc: 0.997 | Test acc:0.114
Epoch:12 | Train loss: 1.46 | Train acc: 0.997 | Test acc:0.118
Epoch:13 | Train loss: 1.46 | Train acc: 0.997 | Test acc:0.121
Without Dropout & Batch Normalization & Adversarial Training


/home/root/adversarial/Gal/models.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:422: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

Epoch:0 | Train loss: 31.94 | Train acc: 0.831 | Test acc:0.088
Epoch:1 | Train loss: 16.21 | Train acc: 0.837 | Test acc:0.110
Epoch:2 | Train loss: 8.33 | Train acc: 0.842 | Test acc:0.083
Epoch:3 | Train loss: 4.46 | Train acc: 0.843 | Test acc:0.097
Epoch:4 | Train loss: 2.67 | Train acc: 0.844 | Test acc:0.090
Epoch:5 | Train loss: 1.91 | Train acc: 0.846 | Test acc:0.108
Epoch:6 | Train loss: 1.61 | Train acc: 0.846 | Test acc:0.105
Epoch:7 | Train loss: 1.51 | Train acc: 0.846 | Test acc:0.108
Epoch:8 | Train loss: 1.47 | Train acc: 0.846 | Test acc:0.113
Epoch:9 | Train loss: 1.47 | Train acc: 0.847 | Test acc:0.092
Epoch:10 | Train loss: 1.46 | Train acc: 0.847 | Test acc:0.103
Epoch:11 | Train loss: 1.46 | Train acc: 0.847 | Test acc:0.116
Epoch:12 | Train loss: 1.46 | Train acc: 0.849 | Test acc:0.115
Epoch:13 | Train loss: 1.46 | Train acc: 0.848 | Test acc:0.109


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnin

...

/home/root/adversarial/Gal/models.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:422: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

Epoch:0 | Train loss: 59.59 | Train acc: 0.812 | Test acc:0.095
Epoch:1 | Train loss: 52.07 | Train acc: 0.831 | Test acc:0.087
Epoch:2 | Train loss: 45.61 | Train acc: 0.840 | Test acc:0.072
Epoch:3 | Train loss: 40.02 | Train acc: 0.844 | Test acc:0.074
Epoch:4 | Train loss: 35.17 | Train acc: 0.846 | Test acc:0.084
Epoch:5 | Train loss: 30.95 | Train acc: 0.846 | Test acc:0.068
Epoch:6 | Train loss: 27.27 | Train acc: 0.849 | Test acc:0.079
Epoch:7 | Train loss: 24.06 | Train acc: 0.849 | Test acc:0.091
Epoch:8 | Train loss: 21.25 | Train acc: 0.849 | Test acc:0.081
Epoch:9 | Train loss: 18.80 | Train acc: 0.849 | Test acc:0.077
Epoch:10 | Train loss: 16.65 | Train acc: 0.849 | Test acc:0.080
Epoch:11 | Train loss: 14.77 | Train acc: 0.850 | Test acc:0.081
Epoch:12 | Train loss: 13.11 | Train acc: 0.850 | Test acc:0.085
Epoch:13 | Train loss: 11.66 | Train acc: 0.849 | Test acc:0.095
Without Dropout & Batch Normalization


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 59.40 | Train acc: 0.943 | Test acc:0.119
Epoch:1 | Train loss: 52.95 | Train acc: 0.971 | Test acc:0.113
Epoch:2 | Train loss: 47.28 | Train acc: 0.984 | Test acc:0.097
Epoch:3 | Train loss: 42.26 | Train acc: 0.989 | Test acc:0.102
Epoch:4 | Train loss: 37.80 | Train acc: 0.993 | Test acc:0.085
Epoch:5 | Train loss: 33.84 | Train acc: 0.994 | Test acc:0.101
Epoch:6 | Train loss: 30.31 | Train acc: 0.996 | Test acc:0.088
Epoch:7 | Train loss: 27.15 | Train acc: 0.996 | Test acc:0.092
Epoch:8 | Train loss: 24.33 | Train acc: 0.997 | Test acc:0.079
Epoch:9 | Train loss: 21.81 | Train acc: 0.997 | Test acc:0.090
Epoch:10 | Train loss: 19.56 | Train acc: 0.998 | Test acc:0.082
Epoch:11 | Train loss: 17.54 | Train acc: 0.998 | Test acc:0.083
Epoch:12 | Train loss: 15.74 | Train acc: 0.998 | Test acc:0.075
Epoch:13 | Train loss: 14.12 | Train acc: 0.998 | Test acc:0.085
Without Dropout & Batch Normalization & Adversarial Training


/home/root/adversarial/Gal/models.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
/home/root/adversarial/Gal/models.py:422: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  sample_Y_train = Y_train[[ind]]
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already activ

In [1]:
entropy_val_MLPat_10

NameError: name 'entropy_val_MLPat_10' is not defined

## Figure 3(a) plots

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(entropy_val_MLP_1, normed=True, bins=300, label = '1')
plt.hist(entropy_val_MLP_5, normed=True, bins=300, label = '5')
plt.hist(entropy_val_MLP_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble - MNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(entropy_val_MLPr_1, normed=True, bins=300, label = '1')
plt.hist(entropy_val_MLPr_5, normed=True, bins=300, label = '5')
plt.hist(entropy_val_MLPr_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble+R - MNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(entropy_val_MLPat_1, normed=True, bins=300, label = '1')
plt.hist(entropy_val_MLPat_5, normed=True, bins=300, label = '5')
plt.hist(entropy_val_MLPat_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble+AT - MNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(entropy_val_MLPmc_1, normed=True, bins=300, label = '1')
plt.hist(entropy_val_MLPmc_5, normed=True, bins=300, label = '5')
plt.hist(entropy_val_MLPmc_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("MC Dropout 0.1 - MNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(Nentropy_val_MLP_1, normed=True, bins=300, label = '1')
plt.hist(Nentropy_val_MLP_5, normed=True, bins=300, label = '5')
plt.hist(Nentropy_val_MLP_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble - notMNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(Nentropy_val_MLPr_1, normed=True, bins=300, label = '1')
plt.hist(Nentropy_val_MLPr_5, normed=True, bins=300, label = '5')
plt.hist(Nentropy_val_MLPr_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble+R - notMNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(Nentropy_val_MLPat_1, normed=True, bins=300, label = '1')
plt.hist(Nentropy_val_MLPat_5, normed=True, bins=300, label = '5')
plt.hist(Nentropy_val_MLPat_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("Ensemble+AT - notMNIST")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(Nentropy_val_MLPmc_1, normed=True, bins=300, label = '1')
plt.hist(Nentropy_val_MLPmc_5, normed=True, bins=300, label = '5')
plt.hist(Nentropy_val_MLPmc_10, normed=True, bins=300, label = '10')
plt.xlabel('Entropy values')
plt.title("MC Dropout 0.1 - notMNIST")
plt.legend()
plt.show()

In [ ]:
# entropy1_hist_list = [entropy_val_MLP_1,entropy_val_MLPr_1,entropy_val_MLPat_1,entropy_val_MLPmc_1,Nentropy_val_MLP_1,Nentropy_val_MLPr_1,Nentropy_val_MLPat_1,Nentropy_val_MLPmc_1]
# entropy5_hist_list = [entropy_val_MLP_5,entropy_val_MLPr_5,entropy_val_MLPat_5,entropy_val_MLPmc_5,Nentropy_val_MLP_5,Nentropy_val_MLPr_5,Nentropy_val_MLPat_5,Nentropy_val_MLPmc_5]
# entropy10_hist_list = [entropy_val_MLP_10,entropy_val_MLPr_10,entropy_val_MLPat_10,entropy_val_MLPmc_10,Nentropy_val_MLP_10,Nentropy_val_MLPr_10,Nentropy_val_MLPat_10,Nentropy_val_MLPmc_10]
# title_list=["Ensemble - MNIST","Ensemble+R - MNIST","Ensemble+AT - MNIST","MC Dropout 0.1 - MNIST","Ensemble - notMNIST","Ensemble+R - notMNIST","Ensemble+AT - notMNIST","MC Dropout 0.1 - notMNIST"]

# import matplotlib.pyplot as plt
# %matplotlib inline

# for i in range(1, 7):
#     plt.subplot(2, 3, i)
#     plt.hist(entropy1_hist_list[i], normed=True, bins=300, label = '1')
#     plt.hist(entropy5_hist_list[i], normed=True, bins=300, label = '5')
#     plt.hist(entropy10_hist_list[i], normed=True, bins=300, label = '10')
#     plt.xlabel('Entropy values')
#     plt.title(title_list[i])
#     plt.legend()
# #     plt.show()

In [ ]:
# entropy1_hist_list = [entropy_val_MLP_1,entropy_val_MLPr_1,entropy_val_MLPat_1]
# title_list=["Ensemble - MNIST","Ensemble+R - MNIST","Ensemble+AT - MNIST","MC Dropout 0.1 - MNIST","Ensemble - notMNIST","Ensemble+R - notMNIST","Ensemble+AT - notMNIST","MC Dropout 0.1 - notMNIST"]

# import matplotlib.pyplot as plt
# %matplotlib inline


# for i in range(0,len(entropy1_hist_list)):
#     plt.hist(entropy1_hist_list[i], normed=True, bins=300, label = '1')
# #     plt.xlim(-0.5,5)
#     plt.xlabel('Entropy values')
#     plt.title(title_list[i])
#     plt.legend()
#     plt.show()

In [ ]:
# # Fig 3a test

# from matplotlib.pyplot import figure
# figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')

# entropy1_hist_list = [entropy_val_MLP_1,entropy_val_MLPr_1,entropy_val_MLPat_1]
# title_list=["Ensemble - MNIST","Ensemble+R - MNIST","Ensemble+AT - MNIST","MC Dropout 0.1 - MNIST","Ensemble - notMNIST","Ensemble+R - notMNIST","Ensemble+AT - notMNIST","MC Dropout 0.1 - notMNIST"]

# import matplotlib.pyplot as plt
# %matplotlib inline

# # for i in range(1, 7):
# # #     plt.subplot(2, 3, i)
# #     plt.hist(entropy1_hist_list[i-1], normed=True, bins=300, label = '1')
# #     plt.xlim(-0.5,5)
# #     plt.xlabel('Entropy values')
# #     plt.title(title_list[i-1])
# #     plt.legend()
# #     plt.show()

# plt.hist(entropy_val_MLP_1, normed=True, bins=300, label = '1')
# # plt.xlim(-0.5,5)
# # plt.ylim(0,6)
# plt.xlabel('Entropy values')
# plt.title(title_list[0])
# plt.legend()
# plt.show()

# plt.hist(entropy_val_MLPr_1, normed=True, bins=300, label = '1')
# # plt.xlim(-0.5,5)
# # plt.ylim(0,6)
# plt.xlabel('Entropy values')
# plt.title(title_list[0])
# plt.legend()
# plt.show()

# plt.hist(entropy_val_MLPat_1, normed=True, bins=300, label = '1')
# # plt.xlim(-0.5,5)
# # plt.ylim(0,6)
# plt.xlabel('Entropy values')
# plt.title(title_list[0])
# plt.legend()
# plt.show()

## Figure 6 plots

In [ ]:
# Fig 6 test

import matplotlib.pyplot as plt
%matplotlib inline

confi_complete_list = [confi_MLP_1, confi_MLPr_1, confi_MLPat_1, confi_MLPmc_1, Nconfi_MLP_1, Nconfi_MLPr_1, Nconfi_MLPat_1, Nconfi_MLPmc_1]
acc_complete_list = [acc_MLP_1, acc_MLPr_1, acc_MLPat_1, acc_MLPmc_1, Nacc_MLP_1, Nacc_MLPr_1, Nacc_MLPat_1, Nacc_MLPmc_1]

plt.scatter(confi_complete_list, acc_complete_list)
plt.xlabel('Confidence')
plt.ylabel('Accuracy')
# plt.legend()
plt.show()